<a href="https://colab.research.google.com/github/Datamin-D/Projects/blob/main/%5BLG_data_con%5D_EDA_%26_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Loading Data 

In [ ]:
#connecting gdrive

from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
#libraries
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
color = sns.color_palette()


In [ ]:
# loading data
sample_submission = pd.read_csv('/gdrive/MyDrive/대회, 프로젝트/LG data con/sample_submission.csv')
test_err = pd.read_csv('/gdrive/MyDrive/대회, 프로젝트/LG data con/test_err_data.csv')
test_quality = pd.read_csv('/gdrive/MyDrive/대회, 프로젝트/LG data con/test_quality_data.csv')
train_err = pd.read_csv('/gdrive/MyDrive/대회, 프로젝트/LG data con/train_err_data.csv')
train_problem = pd.read_csv('/gdrive/MyDrive/대회, 프로젝트/LG data con/train_problem_data.csv')
train_quality = pd.read_csv('/gdrive/MyDrive/대회, 프로젝트/LG data con/train_quality_data.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 2.Preprocessing

## train_err, test_err

In [ ]:
train_err

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0
...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,15,1
16554659,24999,20201130172625,model_3,05.15.2138,16,1
16554660,24999,20201130172625,model_3,05.15.2138,4,0
16554661,24999,20201130172631,model_3,05.15.2138,4,0


In [ ]:
# # time 컬럼 전처리 #datetime분석가능한 형태로.

# # pandas datetime을 활용할 수 있는 형태로 바꿔주기
# def divide_time(time):
#   time = str(time)
#   Year = time[:4]
#   Month = time[4:6]
#   Day = time[6:8]
#   Hour = time[8:10]
#   minute = time[10:12]
#   second = time[12:]
#   #print(Year+"-"+Month+"-"+Day+" "+Hour+":"+minute+":"+second)
#   return Year+"-"+Month+"-"+Day+" "+Hour+":"+minute+":"+second
#train_err['parsed_time']= train_err['time'].map(lambda x: divide_time(x))
# 이럴필요 없음. pd.date_time(df['컬럼'],format = '') 쓰면됨.


#train
train_err
train_err['parsed_time'] = pd.to_datetime(train_err['time'],format ='%Y%m%d%H%M%S') # format으로 datetime인지가능
train_err.drop(['time'],axis = 1,inplace = True)
#test도 
test_err
test_err['parsed_time'] = pd.to_datetime(test_err['time'],format ='%Y%m%d%H%M%S') 
test_err.drop(['time'],axis = 1,inplace = True)


In [ ]:
test_err.drop(['time'],axis = 1,inplace = True)

KeyError: ignored

In [ ]:
train_err

## train_quality, test_quality

In [ ]:
train_quality

In [ ]:
#time 컬럼 전처리

#train

train_quality['parsed_time'] = pd.to_datetime(train_quality['time'],format ='%Y%m%d%H%M%S') # format으로 datetime인지가능
train_quality.drop(['time'],axis = 1,inplace = True)
#test도 

test_quality['parsed_time'] = pd.to_datetime(test_quality['time'],format ='%Y%m%d%H%M%S') 
test_quality.drop(['time'],axis = 1,inplace = True)


##  train_problem

In [ ]:
#time 컬럼 전처리
train_problem['parsed_time'] = pd.to_datetime(train_problem['time'],format = "%Y%m%d%H%M%S")
train_problem.drop(['time'],axis = 1,inplace = True)
train_problem

#3.EDA

-데이터 합쳐야하나..? 뭘? 

-결측치도 확인

##train_err

In [ ]:
train_err
#시스템에 발생한 에러 로그

#errtype? errcode? 차이?
#fwver는? Firmware Version인듯.
#model_nm? Q 모델 네임. 무슨 모델?
#시스템에 발생한 에러 로그

#errtype? errcode? 차이?
#fwver는? Firmware Version인듯.
#model_nm? Q 모델 네임. 무슨 모델?

### user 몇명?

In [ ]:
err_user = train_err['user_id'].value_counts() #Series.value_counts() 하면 유일한 값 별로 세어줌
err_user
#15000명의 유저

In [ ]:
train_err['user_id'].value_counts().max()
#최대 222186건의 에러 로그 발견.최소는 물론1

In [ ]:
err_user.sort_values(ascending = False,inplace = True)
err_user.iloc[:100]

In [ ]:
# 유저별 오류비율 

err_user

sns.kdeplot(err_user)
plt.title("Iris 데이터 중, 꽃잎의 길이에 대한 Kernel Density Plot")
plt.show()

In [ ]:
sns.distplot(err_user, kde=True, rug=True)
plt.title("유저별 에러 로그 빈도")

### model_nm

In [ ]:
err_model = train_err['model_nm'].value_counts()
err_model

In [ ]:
sns.barplot(x = err_model.index,y = err_model.values)
plt.title("Error Frequency by Models")
plt.xticks(rotation='vertical')
plt.ylabel('Count', fontsize=12)
plt.xlabel('Models', fontsize=12)
plt.show()

In [ ]:
# model0,1,2,3,4 모델은 오류 빈도가 높은편. 

### fwver

In [ ]:
err_fwver = train_err['fwver'].value_counts()
err_fwver

In [ ]:
#시각화 
plt.figure(figsize=(12,8))
sns.barplot(x=err_fwver.index,y = err_fwver.values, color=color[5]) 
plt.ylabel('Count', fontsize=12)
plt.xlabel('Firmware Version', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Errors by Firmware Version", fontsize=15)
plt.show()

--> 특정 펌웨어 버젼에서 오류가 자주 발견됨을 볼 수 있음.

### errtype

In [ ]:
err_errtype = train_err['errtype'].value_counts()
err_errtype

In [ ]:
#시각화 
plt.figure(figsize=(12,8))
sns.barplot(x=err_errtype.index,y = err_errtype.values, color=color[1]) 
plt.ylabel('Count', fontsize=12)
plt.xlabel('Error Type', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Errors by Error Type", fontsize=15)
plt.show()

In [ ]:
# 다른 컬럼과 섞어봐도 좋을듯. 어떤 error type이 많이 발생하는지 진단.

errtype, user_id

errtype, model_nm

errtype, fwver

In [ ]:
train_quality
#시스템 퀄리티 로그
# quality?

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,1,0,0,0,0,17,0,0
828620,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0
828621,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,3,0,0,0,0,17,0,0
828622,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0


In [ ]:
train_problem
#사용자 불만 및 불만이 접수된 시간

,user_id,time
0,19224,20201102200000
1,23664,20201116140000
2,15166,20201114130000
3,12590,20201108210000
4,15932,20201103210000
...,...,...
5424,20167,20201125120000
5425,16270,20201110120000
5426,19114,20201106230000
5427,21505,20201104110000


In [ ]:
sample_submission

,user_id,problem
0,30000,0
1,30001,0
2,30002,0
3,30003,0
4,30004,0
...,...,...
14994,44994,0
14995,44995,0
14996,44996,0
14997,44997,0
